<a href="https://colab.research.google.com/github/zkc1031/BadmintonNetTouch/blob/main/main_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Googleドライブ連携
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#必要ライブラリのインストール


# 感嘆符(!)をつけると、ターミナルコマンドとして実行される
!pip install "numpy<2.0" mediapipe==0.10.9 ultralytics torch
# protobufとtensorflowは、Colabにプリインストールされている互換性のあるバージョンをそのまま使うのが安定します

In [4]:
#フレームラベル付けシステム

# ライブラリのインポート
import os
import shutil
import random
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image

# --- 設定 ---
# ラベル付け候補の画像が入っているフォルダ
SOURCE_DIR = '/content/drive/MyDrive/Badminton_Research/datasets/frames_to_label/'

# 分類後の画像を保存する親フォルダ
DEST_DIR_BASE = '/content/drive/MyDrive/Badminton_Research/datasets/labeled_data/'

# クラス名と、対応するフォルダ名
CLASSES = ['net_touch', 'over_net', 'normal']

# --- 関数の定義 ---

# 各クラスの保存先フォルダを作成する
for class_name in CLASSES:
    os.makedirs(os.path.join(DEST_DIR_BASE, class_name), exist_ok=True)
# スキップした画像用のフォルダも作成
os.makedirs(os.path.join(DEST_DIR_BASE, 'skipped'), exist_ok=True)

# SOURCE_DIRが存在するかチェック
if not os.path.exists(SOURCE_DIR):
    print(f"Error: Source directory not found at {SOURCE_DIR}")
    print("Please check if the path is correct and the directory exists in your Google Drive.")
else:
    # ファイルリストを取得し、シャッフルする
    image_files = [f for f in os.listdir(SOURCE_DIR) if f.endswith(('.png', '.jpg', '.jpeg'))]
    random.shuffle(image_files)

    # 現在のインデックスを管理する
    current_index = 0

    # 画像表示用のウィジェット
    image_widget = widgets.Image(format='png', width=800)

    # ボタンウィジェットの作成
    buttons = []
    for label in CLASSES + ['skip']:
        button = widgets.Button(description=label)
        buttons.append(button)

    # ボタンがクリックされたときの処理を定義
    def on_button_clicked(b):
        global current_index

        # 古い画像とボタンをクリア
        clear_output(wait=True)

        # 選択されたラベルに基づいてファイルを移動
        label = b.description
        if current_index < len(image_files):
            img_name = image_files[current_index]
            source_path = os.path.join(SOURCE_DIR, img_name)
            dest_path = os.path.join(DEST_DIR_BASE, label, img_name)

            try:
                shutil.move(source_path, dest_path)
                print(f"Moved '{img_name}' to '{label}' folder.")
            except Exception as e:
                print(f"Error moving file: {e}")

        # 次の画像へ
        current_index += 1
        show_next_image()

    # 各ボタンにクリックイベントを登録
    for button in buttons:
        button.on_click(on_button_clicked)

    # 次の画像を表示する関数
    def show_next_image():
        global current_index
        if current_index < len(image_files):
            img_name = image_files[current_index]
            img_path = os.path.join(SOURCE_DIR, img_name)

            # 画像を読み込んで表示
            with open(img_path, 'rb') as f:
                image_widget.value = f.read()

            # 進捗を表示
            progress_label = widgets.Label(f"Image {current_index + 1} / {len(image_files)}: {img_name}")

            # 画像とボタンをまとめて表示
            display(progress_label, image_widget, widgets.HBox(buttons))
        else:
            print("全ての画像の分類が完了しました！お疲れ様でした。")

    # 最初の画像を表示して開始
    if image_files:
        show_next_image()
    else:
        print("Source directory is empty or does not contain supported image files (.png, .jpg, .jpeg).")

Error: Source directory not found at /content/drive/MyDrive/Badminton_Research/datasets/frames_to_label/
Please check if the path is correct and the directory exists in your Google Drive.


In [ ]:
# 最初の画像を表示してラベリングを開始
show_next_image()

In [5]:
#抽出用コード

# 必要なライブラリのインポート
import cv2
import mediapipe as mp
import os
import ntpath
from scipy.spatial import distance as dist
import numpy as np

# MediaPipeの新しいAPIのためのインポート
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# --- 設定 ---
# あなたがGoogle Driveにアップロードしたテスト用動画のパスに書き換えてください
VIDEO_PATH = '/content/drive/MyDrive/Badminton_Research/videos/test_video_01.mp4'

# 抽出した画像を保存するフォルダのパス
OUTPUT_DIR_FRAMES = '/content/drive/MyDrive/Badminton_Research/datasets/frames_to_label/'

# 物体検出モデルのパス
MODEL_PATH = '/content/drive/MyDrive/Badminton_Research/models/efficientdet_lite0.tflite'

# 判定の感度設定
DISTANCE_THRESHOLD = 150
DISPLAY_WIDTH = 1280 # Colabでは表示しないが、計算に使うため残す

# --- MediaPipeの準備 ---
mp_pose = mp.solutions.pose

# --- 補助関数 ---
def get_bbox_center(bbox):
    return (bbox.origin_x + bbox.width // 2, bbox.origin_y + bbox.height // 2)

# --- メイン処理 ---
def extract_important_frames():
    # フォルダの準備
    os.makedirs(OUTPUT_DIR_FRAMES, exist_ok=True)

    # ファイル存在チェック
    if not os.path.exists(VIDEO_PATH):
        print(f"エラー: 動画ファイルが見つかりません: {VIDEO_PATH}")
        return
    if not os.path.exists(MODEL_PATH):
        print(f"エラー: モデルファイルが見つかりません: {MODEL_PATH}")
        return

    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print(f"エラー: 動画を開けませんでした: {VIDEO_PATH}")
        return

    # 最初のフレームでネットのROIを選択させる（ローカル実行時と同じUIが一時的に表示されます）
    ret, first_frame = cap.read()
    if not ret: return

    # この部分はColabでは直接表示されませんが、ROI選択のために必要です
    # 注意：Colabで実行すると、このUIは表示されず、デフォルト値(0,0,0,0)になる可能性があります。
    # その場合は、ROI座標を直接コードに書き込む必要があります。
    # 今回はまず、このまま実行してみましょう。
    cv2.putText(first_frame, "Select Net ROI, then press ENTER", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    net_roi = cv2.selectROI("Select Net ROI (in a temporary local window)", first_frame, fromCenter=False)
    cv2.destroyAllWindows()
    # selectROIがColabでうまく動かない場合、以下の行を有効にして手動で設定
    # net_roi = (x, y, w, h) # 例: net_roi = (600, 400, 700, 150)

    if net_roi == (0, 0, 0, 0):
        print("ROIが選択されませんでした。")
        return

    rx, ry, rw, rh = net_roi
    net_roi_center = (rx + rw // 2, ry + rh // 2)
    (h, w) = first_frame.shape[:2]

    video_filename = ntpath.basename(VIDEO_PATH).split('.')[0]
    frame_count = 0
    saved_count = 0

    # MediaPipeモデルの準備
    base_options = python.BaseOptions(model_asset_path=MODEL_PATH)
    object_options = vision.ObjectDetectorOptions(base_options=base_options, score_threshold=0.5, category_allowlist=["sports racket"])
    object_detector = vision.ObjectDetector.create_from_options(object_options)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print("--- フレーム抽出を開始します ---")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break

            frame_count += 1
            if frame_count % 30 == 0: # 30フレームごとに進捗を表示
                print(f"Processing frame {frame_count} / {total_frames}...")

            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            pose_results = pose.process(mp_image.numpy_view())
            detection_result = object_detector.detect(mp_image)

            if pose_results.pose_landmarks and detection_result.detections:
                racket_bbox = detection_result.detections[0].bounding_box
                racket_center = get_bbox_center(racket_bbox)

                landmarks = pose_results.pose_landmarks.landmark
                left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
                right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]

                left_wrist_pos = (int(left_wrist.x * w), int(left_wrist.y * h))
                right_wrist_pos = (int(right_wrist.x * w), int(right_wrist.y * h))

                dist_to_left = dist.euclidean(racket_center, left_wrist_pos)
                dist_to_right = dist.euclidean(racket_center, right_wrist_pos)

                racket_hand_pos = left_wrist_pos if dist_to_left < dist_to_right else right_wrist_pos

                d = dist.euclidean(racket_hand_pos, net_roi_center)

                if d < DISTANCE_THRESHOLD + (rw+rh)/2:
                    save_filename = f'{video_filename}_frame_{frame_count}.png'
                    save_path = os.path.join(OUTPUT_DIR_FRAMES, save_filename)
                    cv2.imwrite(save_path, frame)
                    saved_count += 1
                    print(f"  -> Frame {frame_count}: 接近を検知。画像を保存しました。({saved_count}枚目)")

    cap.release()
    print(f"--- 抽出完了 ---")
    print(f"合計 {saved_count} 枚の画像を '{OUTPUT_DIR_FRAMES}' に保存しました。")